# 🚀 TrustformeRS Interactive Tutorial

Welcome to the comprehensive TrustformeRS tutorial! This notebook will guide you through all the major features of TrustformeRS, the high-performance machine learning library written in Rust.

## 📚 What You'll Learn

1. **Basic Pipeline Usage** - Text classification, generation, and Q&A
2. **Advanced Features** - Batching, streaming, and optimization
3. **Model Management** - Loading, configuring, and comparing models
4. **Performance Optimization** - JIT compilation, caching, and profiling
5. **Production Features** - Serving, monitoring, and deployment

## 🛠️ Setup

First, let's install TrustformeRS and set up our environment:

In [ ]:
# Install TrustformeRS (in a real environment)
# !pip install trustformers

# For this tutorial, we'll use mock implementations
import json
import time
import random
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np

print("✅ TrustformeRS environment ready!")
print("📖 This tutorial uses simulated TrustformeRS functionality for demonstration")

## 🎯 Chapter 1: Basic Pipeline Usage

Let's start with the fundamental building blocks of TrustformeRS - pipelines!

In [ ]:
# Mock TrustformeRS classes for tutorial purposes
@dataclass
class ClassificationResult:
    label: str
    score: float
    all_scores: List[Dict[str, Any]]

@dataclass
class GenerationResult:
    generated_text: str
    prompt: str
    
@dataclass
class QAResult:
    answer: str
    confidence: float
    context: str
    question: str

class MockTextClassificationPipeline:
    def __init__(self, model_name: str):
        self.model_name = model_name
        print(f"🤖 Loading {model_name}...")
        time.sleep(0.5)  # Simulate loading
        print("✅ Model loaded successfully!")
    
    def __call__(self, text: str) -> ClassificationResult:
        # Simple sentiment analysis simulation
        positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'love', 'fantastic']
        negative_words = ['bad', 'terrible', 'awful', 'hate', 'horrible', 'worst']
        
        text_lower = text.lower()
        positive_count = sum(1 for word in positive_words if word in text_lower)
        negative_count = sum(1 for word in negative_words if word in text_lower)
        
        if positive_count > negative_count:
            label, score = "POSITIVE", 0.85 + min(positive_count * 0.1, 0.15)
        elif negative_count > positive_count:
            label, score = "NEGATIVE", 0.85 + min(negative_count * 0.1, 0.15)
        else:
            label, score = "NEUTRAL", 0.55
        
        return ClassificationResult(
            label=label,
            score=score,
            all_scores=[
                {"label": "POSITIVE", "score": score if label == "POSITIVE" else 1.0 - score},
                {"label": "NEGATIVE", "score": score if label == "NEGATIVE" else 1.0 - score},
            ]
        )
    
    def batch(self, texts: List[str]) -> List[ClassificationResult]:
        print(f"🔄 Processing batch of {len(texts)} texts...")
        return [self(text) for text in texts]

class MockTextGenerationPipeline:
    def __init__(self, model_name: str):
        self.model_name = model_name
        print(f"🤖 Loading {model_name}...")
        time.sleep(0.5)
        print("✅ Model loaded successfully!")
    
    def __call__(self, prompt: str, max_length: int = 50) -> GenerationResult:
        continuations = [
            "is an incredible advancement in machine learning technology.",
            "represents the future of artificial intelligence and natural language processing.",
            "demonstrates the power of modern deep learning architectures.",
            "shows how transformer models can understand and generate human-like text."
        ]
        
        continuation = continuations[len(prompt) % len(continuations)]
        generated = f"{prompt} {continuation}"
        
        # Simulate word limit
        words = generated.split()
        if len(words) > max_length:
            generated = " ".join(words[:max_length]) + "..."
        
        return GenerationResult(generated_text=generated, prompt=prompt)

def pipeline(task: str, model: str = None):
    """Factory function to create pipelines"""
    default_models = {
        "text-classification": "distilbert-base-uncased-finetuned-sst-2-english",
        "text-generation": "gpt2",
        "question-answering": "distilbert-base-cased-distilled-squad"
    }
    
    model_name = model or default_models.get(task, "default-model")
    
    if task == "text-classification":
        return MockTextClassificationPipeline(model_name)
    elif task == "text-generation":
        return MockTextGenerationPipeline(model_name)
    else:
        raise ValueError(f"Task {task} not supported in this tutorial")

print("🔧 TrustformeRS pipeline factory ready!")

### 📊 Text Classification

Let's start with text classification - one of the most common NLP tasks:

In [ ]:
# Create a text classification pipeline
classifier = pipeline("text-classification")

# Test with individual examples
examples = [
    "I love this new transformer library!",
    "This is the worst software I've ever used.",
    "The weather is nice today.",
    "TrustformeRS makes ML so much easier!"
]

print("🔍 Single Text Classification Examples:")
print("=" * 50)

for text in examples:
    result = classifier(text)
    print(f"Text: \"{text}\"")
    print(f"Result: {result.label} ({result.score:.3f})")
    print()

# Visualize results
labels = [classifier(text).label for text in examples]
scores = [classifier(text).score for text in examples]

plt.figure(figsize=(12, 6))
colors = ['green' if label == 'POSITIVE' else 'red' if label == 'NEGATIVE' else 'gray' for label in labels]
bars = plt.bar(range(len(examples)), scores, color=colors, alpha=0.7)
plt.xlabel('Example')
plt.ylabel('Confidence Score')
plt.title('🎯 Text Classification Results')
plt.xticks(range(len(examples)), [f'Ex {i+1}' for i in range(len(examples))])
plt.ylim(0, 1)

# Add value labels on bars
for i, (bar, score, label) in enumerate(zip(bars, scores, labels)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{label}\n{score:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

### 📦 Batch Processing

For production use, batch processing is much more efficient:

In [ ]:
# Batch processing example
batch_texts = [
    "TrustformeRS is amazing for production ML!",
    "I can't figure out how to use this library.",
    "The documentation is comprehensive and helpful.",
    "Performance is incredible compared to Python alternatives.",
    "Installation was a bit tricky on my system."
]

print("📦 Batch Processing Demo:")
print("=" * 30)

# Time the batch processing
start_time = time.time()
batch_results = classifier.batch(batch_texts)
batch_time = time.time() - start_time

# Time individual processing for comparison
start_time = time.time()
individual_results = [classifier(text) for text in batch_texts]
individual_time = time.time() - start_time

print(f"⚡ Batch processing time: {batch_time:.3f}s")
print(f"🐌 Individual processing time: {individual_time:.3f}s")
print(f"🚀 Speedup: {individual_time/batch_time:.1f}x faster")
print()

# Display results in a nice table
print("Results:")
print("-" * 80)
for i, (text, result) in enumerate(zip(batch_texts, batch_results)):
    print(f"{i+1:2d}. {text[:50]:50s} | {result.label:8s} ({result.score:.3f})")

# Visualize batch results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Sentiment distribution
sentiment_counts = {}
for result in batch_results:
    sentiment_counts[result.label] = sentiment_counts.get(result.label, 0) + 1

ax1.pie(sentiment_counts.values(), labels=sentiment_counts.keys(), autopct='%1.1f%%', startangle=90)
ax1.set_title('📊 Sentiment Distribution')

# Confidence scores
ax2.hist([result.score for result in batch_results], bins=10, alpha=0.7, color='skyblue')
ax2.set_xlabel('Confidence Score')
ax2.set_ylabel('Frequency')
ax2.set_title('📈 Confidence Score Distribution')

plt.tight_layout()
plt.show()

### ✍️ Text Generation

Now let's explore text generation capabilities:

In [ ]:
# Create a text generation pipeline
generator = pipeline("text-generation", "gpt2")

# Creative writing prompts
prompts = [
    "The future of artificial intelligence",
    "Once upon a time in a distant galaxy",
    "The benefits of renewable energy include",
    "In the world of machine learning"
]

print("✍️ Text Generation Examples:")
print("=" * 40)

generated_texts = []
for i, prompt in enumerate(prompts):
    print(f"\n{i+1}. Prompt: \"{prompt}\"")
    
    # Generate with different max lengths
    for max_len in [20, 40]:
        result = generator(prompt, max_length=max_len)
        print(f"   Max Length {max_len:2d}: {result.generated_text}")
        generated_texts.append(result.generated_text)

# Analyze generation statistics
word_counts = [len(text.split()) for text in generated_texts]
char_counts = [len(text) for text in generated_texts]

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(range(len(word_counts)), word_counts, color='lightcoral')
plt.xlabel('Generation')
plt.ylabel('Word Count')
plt.title('📊 Generated Text Length (Words)')
plt.xticks(range(len(word_counts)), [f'Gen {i+1}' for i in range(len(word_counts))], rotation=45)

plt.subplot(1, 2, 2)
plt.bar(range(len(char_counts)), char_counts, color='lightblue')
plt.xlabel('Generation')
plt.ylabel('Character Count')
plt.title('📊 Generated Text Length (Characters)')
plt.xticks(range(len(char_counts)), [f'Gen {i+1}' for i in range(len(char_counts))], rotation=45)

plt.tight_layout()
plt.show()

## 🚀 Chapter 2: Advanced Features

Let's explore some of TrustformeRS's advanced capabilities!

### 🎛️ Pipeline Configuration

TrustformeRS provides extensive configuration options for fine-tuning performance:

In [ ]:
# Mock configuration classes
@dataclass
class PipelineConfig:
    batch_size: int = 16
    max_length: int = 512
    device: str = "auto"
    precision: str = "fp16"
    enable_caching: bool = True
    enable_jit: bool = False

@dataclass
class GenerationConfig:
    max_length: int = 50
    temperature: float = 1.0
    top_k: int = 50
    top_p: float = 1.0
    do_sample: bool = True
    num_return_sequences: int = 1

# Create configurations for different use cases
configs = {
    "fast": PipelineConfig(batch_size=32, precision="fp16", enable_jit=True),
    "accurate": PipelineConfig(batch_size=8, precision="fp32", enable_jit=False),
    "memory_efficient": PipelineConfig(batch_size=4, max_length=256, enable_caching=False)
}

print("🎛️ Pipeline Configuration Options:")
print("=" * 40)

for name, config in configs.items():
    print(f"\n{name.upper()} Configuration:")
    for field, value in config.__dict__.items():
        print(f"  {field:15s}: {value}")

# Simulate performance comparison
performance_data = {
    "fast": {"latency": 25, "memory": 1800, "accuracy": 0.92},
    "accurate": {"latency": 45, "memory": 2400, "accuracy": 0.96},
    "memory_efficient": {"latency": 35, "memory": 1200, "accuracy": 0.94}
}

# Visualize configuration trade-offs
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['latency', 'memory', 'accuracy']
titles = ['⚡ Latency (ms)', '💾 Memory (MB)', '🎯 Accuracy']
colors = ['red', 'blue', 'green']

for i, (metric, title, color) in enumerate(zip(metrics, titles, colors)):
    values = [performance_data[config][metric] for config in configs.keys()]
    axes[i].bar(configs.keys(), values, color=color, alpha=0.7)
    axes[i].set_title(title)
    axes[i].set_ylabel('Value')
    
    # Add value labels
    for j, v in enumerate(values):
        axes[i].text(j, v + max(values) * 0.01, f'{v}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n💡 Configuration Tips:")
print("- Use 'fast' for real-time applications")
print("- Use 'accurate' for research and high-quality results")
print("- Use 'memory_efficient' for edge deployment or limited resources")

### 🔄 Streaming Processing

TrustformeRS supports streaming for real-time applications:

In [ ]:
import time
from IPython.display import display, clear_output
import threading

class StreamingGenerator:
    def __init__(self, base_generator):
        self.base_generator = base_generator
    
    def stream_generate(self, prompt: str, max_tokens: int = 20):
        """Simulate streaming text generation"""
        # In real TrustformeRS, this would yield tokens as they're generated
        words = ["Machine", "learning", "is", "revolutionizing", "how", "we", "interact", 
                "with", "technology", "and", "data", "in", "unprecedented", "ways."]
        
        generated_text = prompt
        for i, word in enumerate(words[:max_tokens]):
            if i > 0:
                yield " " + word
            else:
                yield " " + word
            time.sleep(0.2)  # Simulate generation delay

# Create streaming generator
streaming_gen = StreamingGenerator(generator)

print("🌊 Streaming Text Generation Demo:")
print("=" * 35)

prompt = "The future of AI"
print(f"Prompt: '{prompt}'")
print("Generated text will appear word by word...\n")

# Simulate streaming output
current_text = prompt
print(f"Output: {current_text}", end="", flush=True)

for token in streaming_gen.stream_generate(prompt, max_tokens=10):
    current_text += token
    print(token, end="", flush=True)

print("\n\n✅ Streaming complete!")

# Streaming vs Batch comparison
print("\n📊 Streaming vs Batch Processing:")
print("-" * 40)

comparison_data = {
    'Method': ['Streaming', 'Batch'],
    'Time to First Token (ms)': [50, 200],
    'Total Time (ms)': [2000, 1800],
    'Memory Usage (MB)': [800, 1200],
    'User Experience': ['Real-time', 'Wait then see all']
}

for key, values in comparison_data.items():
    print(f"{key:25s}: {str(values[0]):12s} vs {str(values[1]):12s}")

## ⚡ Chapter 3: Performance Optimization

TrustformeRS provides several optimization techniques to maximize performance:

### 🔥 JIT Compilation

Just-in-time compilation can significantly speed up inference:

In [ ]:
# Mock JIT compilation system
class JITOptimizer:
    def __init__(self):
        self.compilation_cache = {}
        self.enabled = False
    
    def enable(self):
        self.enabled = True
        print("🔥 JIT compilation enabled")
    
    def compile_pipeline(self, pipeline_type: str):
        if pipeline_type not in self.compilation_cache:
            print(f"⚙️ Compiling {pipeline_type} pipeline...")
            time.sleep(1)  # Simulate compilation time
            self.compilation_cache[pipeline_type] = True
            print(f"✅ {pipeline_type} pipeline compiled")
        else:
            print(f"💾 Using cached compilation for {pipeline_type}")
    
    def benchmark_inference(self, pipeline_type: str, num_runs: int = 100):
        """Benchmark inference with and without JIT"""
        # Simulate performance improvements
        base_latency = random.uniform(80, 120)  # ms
        
        if self.enabled and pipeline_type in self.compilation_cache:
            jit_latency = base_latency * 0.6  # 40% improvement
            return {
                'without_jit': base_latency,
                'with_jit': jit_latency,
                'speedup': base_latency / jit_latency
            }
        else:
            return {
                'without_jit': base_latency,
                'with_jit': None,
                'speedup': 1.0
            }

# Performance optimization demo
print("🚀 JIT Compilation Performance Demo")
print("=" * 40)

jit_optimizer = JITOptimizer()

# Benchmark without JIT
print("\n1. Baseline Performance (No JIT):")
baseline_results = {}
pipeline_types = ['text-classification', 'text-generation', 'question-answering']

for pipeline_type in pipeline_types:
    result = jit_optimizer.benchmark_inference(pipeline_type)
    baseline_results[pipeline_type] = result['without_jit']
    print(f"  {pipeline_type:20s}: {result['without_jit']:.1f} ms")

# Enable JIT and compile
print("\n2. Enabling JIT Compilation:")
jit_optimizer.enable()

for pipeline_type in pipeline_types:
    jit_optimizer.compile_pipeline(pipeline_type)

# Benchmark with JIT
print("\n3. Performance with JIT:")
jit_results = {}
speedups = {}

for pipeline_type in pipeline_types:
    result = jit_optimizer.benchmark_inference(pipeline_type)
    jit_results[pipeline_type] = result['with_jit']
    speedups[pipeline_type] = result['speedup']
    print(f"  {pipeline_type:20s}: {result['with_jit']:.1f} ms ({result['speedup']:.1f}x speedup)")

# Visualize performance improvements
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Latency comparison
x = np.arange(len(pipeline_types))
width = 0.35

baseline_values = [baseline_results[pt] for pt in pipeline_types]
jit_values = [jit_results[pt] for pt in pipeline_types]

ax1.bar(x - width/2, baseline_values, width, label='Without JIT', color='lightcoral')
ax1.bar(x + width/2, jit_values, width, label='With JIT', color='lightgreen')

ax1.set_xlabel('Pipeline Type')
ax1.set_ylabel('Latency (ms)')
ax1.set_title('⚡ JIT Performance Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels([pt.replace('-', '\n') for pt in pipeline_types])
ax1.legend()

# Speedup visualization
speedup_values = [speedups[pt] for pt in pipeline_types]
bars = ax2.bar(pipeline_types, speedup_values, color='gold')
ax2.set_xlabel('Pipeline Type')
ax2.set_ylabel('Speedup Factor')
ax2.set_title('🚀 JIT Speedup Factors')
ax2.set_xticklabels([pt.replace('-', '\n') for pt in pipeline_types])

# Add speedup labels
for bar, speedup in zip(bars, speedup_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
             f'{speedup:.1f}x', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n💡 Average speedup: {np.mean(speedup_values):.1f}x")
print("🎯 JIT compilation provides significant performance improvements!")

### 💾 Caching System

TrustformeRS includes a sophisticated caching system to avoid redundant computations:

In [ ]:
# Mock caching system
class TrustformeRSCache:
    def __init__(self, max_size: int = 1000):
        self.cache = {}
        self.max_size = max_size
        self.hits = 0
        self.misses = 0
    
    def get(self, key: str):
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        else:
            self.misses += 1
            return None
    
    def put(self, key: str, value):
        if len(self.cache) >= self.max_size:
            # Simple LRU: remove oldest item
            oldest_key = next(iter(self.cache))
            del self.cache[oldest_key]
        self.cache[key] = value
    
    def get_stats(self):
        total = self.hits + self.misses
        hit_rate = self.hits / total if total > 0 else 0
        return {
            'hits': self.hits,
            'misses': self.misses,
            'hit_rate': hit_rate,
            'cache_size': len(self.cache)
        }

class CachedPipeline:
    def __init__(self, base_pipeline, cache_size: int = 100):
        self.base_pipeline = base_pipeline
        self.cache = TrustformeRSCache(cache_size)
    
    def __call__(self, text: str):
        # Use text hash as cache key
        cache_key = f"classify_{hash(text)}"
        
        # Check cache first
        cached_result = self.cache.get(cache_key)
        if cached_result is not None:
            return cached_result
        
        # Compute result and cache it
        result = self.base_pipeline(text)
        self.cache.put(cache_key, result)
        return result

# Caching performance demo
print("💾 Caching System Performance Demo")
print("=" * 40)

# Create cached pipeline
base_classifier = pipeline("text-classification")
cached_classifier = CachedPipeline(base_classifier, cache_size=50)

# Test data with some repeated texts
test_texts = [
    "I love TrustformeRS!",
    "This library is amazing",
    "Performance is incredible",
    "I love TrustformeRS!",  # Repeat
    "Documentation is great",
    "This library is amazing",  # Repeat
    "Easy to use and fast",
    "Performance is incredible",  # Repeat
    "Rust is the future",
    "I love TrustformeRS!",  # Repeat
]

print("🔄 Processing texts with caching...")

# Process with timing
processing_times = []
for i, text in enumerate(test_texts):
    start_time = time.time()
    result = cached_classifier(text)
    processing_time = (time.time() - start_time) * 1000  # Convert to ms
    processing_times.append(processing_time)
    
    cache_status = "💾 CACHE HIT" if processing_time < 10 else "🔄 COMPUTED"
    print(f"{i+1:2d}. {text[:30]:30s} | {result.label:8s} | {processing_time:6.1f}ms | {cache_status}")

# Display cache statistics
stats = cached_classifier.cache.get_stats()
print(f"\n📊 Cache Statistics:")
print(f"  Cache hits: {stats['hits']}")
print(f"  Cache misses: {stats['misses']}")
print(f"  Hit rate: {stats['hit_rate']:.1%}")
print(f"  Cache size: {stats['cache_size']} items")

# Visualize cache performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Processing time over requests
ax1.plot(range(1, len(processing_times) + 1), processing_times, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Request Number')
ax1.set_ylabel('Processing Time (ms)')
ax1.set_title('⚡ Processing Time per Request')
ax1.grid(True, alpha=0.3)

# Cache hit/miss visualization
cache_data = ['Hit' if time < 10 else 'Miss' for time in processing_times]
hit_count = cache_data.count('Hit')
miss_count = cache_data.count('Miss')

ax2.pie([hit_count, miss_count], labels=['Cache Hits', 'Cache Misses'], 
        autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'], startangle=90)
ax2.set_title('💾 Cache Hit/Miss Ratio')

plt.tight_layout()
plt.show()

# Performance improvement calculation
avg_cache_hit_time = np.mean([t for t in processing_times if t < 10])
avg_compute_time = np.mean([t for t in processing_times if t >= 10])
speedup = avg_compute_time / avg_cache_hit_time if avg_cache_hit_time > 0 else 1

print(f"\n🚀 Performance Improvement:")
print(f"  Average cache hit time: {avg_cache_hit_time:.1f}ms")
print(f"  Average compute time: {avg_compute_time:.1f}ms")
print(f"  Cache speedup: {speedup:.1f}x faster")

## 🔧 Chapter 4: Model Comparison and Selection

TrustformeRS makes it easy to compare different models and select the best one for your use case:

In [ ]:
# Model comparison framework
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.benchmark_results = {}
    
    def add_model(self, name: str, pipeline):
        self.models[name] = pipeline
        print(f"✅ Added model: {name}")
    
    def benchmark_models(self, test_texts: List[str]):
        print("🏁 Benchmarking models...")
        
        for model_name, pipeline in self.models.items():
            print(f"  Testing {model_name}...")
            
            start_time = time.time()
            results = [pipeline(text) for text in test_texts]
            total_time = time.time() - start_time
            
            # Simulate model-specific performance characteristics
            model_metrics = {
                'DistilBERT': {'latency': 45, 'memory': 1200, 'accuracy': 0.92, 'model_size': 66},
                'RoBERTa': {'latency': 78, 'memory': 2100, 'accuracy': 0.96, 'model_size': 125},
                'ALBERT': {'latency': 52, 'memory': 1400, 'accuracy': 0.94, 'model_size': 12}
            }
            
            base_metrics = model_metrics.get(model_name, model_metrics['DistilBERT'])
            
            self.benchmark_results[model_name] = {
                'avg_latency': base_metrics['latency'] + random.uniform(-5, 5),
                'memory_usage': base_metrics['memory'] + random.uniform(-50, 50),
                'accuracy': base_metrics['accuracy'] + random.uniform(-0.02, 0.02),
                'model_size_mb': base_metrics['model_size'],
                'throughput': len(test_texts) / (base_metrics['latency'] / 1000),
                'results': results
            }
    
    def get_comparison_table(self):
        return self.benchmark_results
    
    def recommend_model(self, priority: str = "balanced"):
        if not self.benchmark_results:
            return None
        
        scores = {}
        
        for model_name, metrics in self.benchmark_results.items():
            if priority == "speed":
                # Higher throughput and lower latency is better
                score = (metrics['throughput'] / 10) + (100 / metrics['avg_latency'])
            elif priority == "accuracy":
                # Higher accuracy is better
                score = metrics['accuracy'] * 100
            elif priority == "memory":
                # Lower memory usage is better
                score = 10000 / metrics['memory_usage']
            else:  # balanced
                # Weighted combination of all factors
                score = (metrics['accuracy'] * 40 + 
                        (metrics['throughput'] / 10) * 20 + 
                        (100 / metrics['avg_latency']) * 20 + 
                        (1000 / metrics['memory_usage']) * 20)
            
            scores[model_name] = score
        
        best_model = max(scores, key=scores.get)
        return best_model, scores

# Model comparison demo
print("🔬 Model Comparison Demo")
print("=" * 30)

# Create different "models" (simulated)
comparator = ModelComparator()

# Add models to compare
models_to_compare = {
    "DistilBERT": pipeline("text-classification", "distilbert-base-uncased"),
    "RoBERTa": pipeline("text-classification", "roberta-base"),
    "ALBERT": pipeline("text-classification", "albert-base")
}

for name, model in models_to_compare.items():
    comparator.add_model(name, model)

# Test texts for comparison
comparison_texts = [
    "TrustformeRS is an excellent ML library",
    "I'm having trouble with this software",
    "The performance improvements are remarkable",
    "Documentation could be better",
    "Rust makes everything so much faster"
]

# Run benchmarks
comparator.benchmark_models(comparison_texts)

# Display results
results = comparator.get_comparison_table()

print("\n📊 Benchmark Results:")
print("-" * 80)
print(f"{'Model':<12} {'Latency(ms)':<12} {'Memory(MB)':<12} {'Accuracy':<10} {'Size(MB)':<10} {'Throughput':<10}")
print("-" * 80)

for model_name, metrics in results.items():
    print(f"{model_name:<12} {metrics['avg_latency']:<12.1f} {metrics['memory_usage']:<12.0f} "
          f"{metrics['accuracy']:<10.3f} {metrics['model_size_mb']:<10.0f} {metrics['throughput']:<10.1f}")

# Visualize comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

models = list(results.keys())
colors = ['lightblue', 'lightgreen', 'lightcoral']

# Latency comparison
latencies = [results[model]['avg_latency'] for model in models]
ax1.bar(models, latencies, color=colors)
ax1.set_title('⚡ Average Latency')
ax1.set_ylabel('Latency (ms)')
for i, v in enumerate(latencies):
    ax1.text(i, v + 1, f'{v:.1f}', ha='center', va='bottom')

# Memory usage comparison
memory_usage = [results[model]['memory_usage'] for model in models]
ax2.bar(models, memory_usage, color=colors)
ax2.set_title('💾 Memory Usage')
ax2.set_ylabel('Memory (MB)')
for i, v in enumerate(memory_usage):
    ax2.text(i, v + 20, f'{v:.0f}', ha='center', va='bottom')

# Accuracy comparison
accuracies = [results[model]['accuracy'] for model in models]
ax3.bar(models, accuracies, color=colors)
ax3.set_title('🎯 Accuracy')
ax3.set_ylabel('Accuracy')
ax3.set_ylim(0.9, 1.0)
for i, v in enumerate(accuracies):
    ax3.text(i, v + 0.002, f'{v:.3f}', ha='center', va='bottom')

# Model size comparison
model_sizes = [results[model]['model_size_mb'] for model in models]
ax4.bar(models, model_sizes, color=colors)
ax4.set_title('📦 Model Size')
ax4.set_ylabel('Size (MB)')
for i, v in enumerate(model_sizes):
    ax4.text(i, v + 2, f'{v:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Get recommendations for different priorities
print("\n🎯 Model Recommendations:")
print("-" * 30)

priorities = ["speed", "accuracy", "memory", "balanced"]
for priority in priorities:
    best_model, scores = comparator.recommend_model(priority)
    print(f"  {priority.upper():8s}: {best_model} (score: {scores[best_model]:.2f})")

print("\n💡 Choose your model based on your requirements:")
print("   - DistilBERT: Good balance of speed and accuracy")
print("   - RoBERTa: Highest accuracy, but slower and more memory-intensive")
print("   - ALBERT: Smallest model size, reasonable performance")

## 🌟 Chapter 5: Real-World Applications

Let's explore how to use TrustformeRS in real-world scenarios:

### 📱 Social Media Sentiment Analysis

A common use case is analyzing social media posts for sentiment:

In [ ]:
# Social media sentiment analysis demo
class SocialMediaAnalyzer:
    def __init__(self):
        self.classifier = pipeline("text-classification", "cardiffnlp/twitter-roberta-base-sentiment")
        self.analysis_results = []
    
    def analyze_posts(self, posts: List[Dict[str, str]]):
        """Analyze a batch of social media posts"""
        print(f"📱 Analyzing {len(posts)} social media posts...")
        
        for post in posts:
            result = self.classifier(post['text'])
            
            analysis = {
                'username': post['username'],
                'text': post['text'],
                'sentiment': result.label,
                'confidence': result.score,
                'timestamp': post.get('timestamp', 'unknown')
            }
            self.analysis_results.append(analysis)
        
        return self.analysis_results
    
    def get_summary_stats(self):
        """Get summary statistics of the analysis"""
        if not self.analysis_results:
            return {}
        
        sentiments = [r['sentiment'] for r in self.analysis_results]
        sentiment_counts = {}
        for sentiment in sentiments:
            sentiment_counts[sentiment] = sentiment_counts.get(sentiment, 0) + 1
        
        avg_confidence = np.mean([r['confidence'] for r in self.analysis_results])
        
        return {
            'total_posts': len(self.analysis_results),
            'sentiment_distribution': sentiment_counts,
            'average_confidence': avg_confidence,
            'most_positive': max(self.analysis_results, key=lambda x: x['confidence'] if x['sentiment'] == 'POSITIVE' else 0),
            'most_negative': max(self.analysis_results, key=lambda x: x['confidence'] if x['sentiment'] == 'NEGATIVE' else 0)
        }

# Sample social media posts (simulated)
sample_posts = [
    {"username": "@tech_enthusiast", "text": "Just tried TrustformeRS and it's blazing fast! 🚀 #rust #ml", "timestamp": "2024-01-15 10:30"},
    {"username": "@data_scientist", "text": "Struggling with memory issues in my ML pipeline 😞", "timestamp": "2024-01-15 11:15"},
    {"username": "@ai_researcher", "text": "The performance improvements in TrustformeRS are incredible! Best decision for our production system.", "timestamp": "2024-01-15 12:00"},
    {"username": "@startup_cto", "text": "Migration to Rust-based ML was painful but worth it", "timestamp": "2024-01-15 13:45"},
    {"username": "@ml_engineer", "text": "Love how TrustformeRS handles large-scale inference ❤️", "timestamp": "2024-01-15 14:20"},
    {"username": "@dev_community", "text": "Documentation is comprehensive and examples are clear 👍", "timestamp": "2024-01-15 15:30"},
    {"username": "@python_dev", "text": "Miss the simplicity of Python libraries sometimes 😔", "timestamp": "2024-01-15 16:10"},
    {"username": "@rust_lover", "text": "This is why Rust is the future of systems programming! 🦀", "timestamp": "2024-01-15 17:00"}
]

print("📱 Social Media Sentiment Analysis Demo")
print("=" * 45)

# Create analyzer and process posts
analyzer = SocialMediaAnalyzer()
results = analyzer.analyze_posts(sample_posts)

# Display individual results
print("\n📊 Individual Post Analysis:")
print("-" * 80)
for result in results:
    sentiment_emoji = "😊" if result['sentiment'] == 'POSITIVE' else "😞" if result['sentiment'] == 'NEGATIVE' else "😐"
    print(f"{result['username']:15s} | {sentiment_emoji} {result['sentiment']:8s} ({result['confidence']:.3f}) | {result['text'][:40]}...")

# Get and display summary statistics
summary = analyzer.get_summary_stats()

print(f"\n📈 Summary Statistics:")
print(f"  Total posts analyzed: {summary['total_posts']}")
print(f"  Average confidence: {summary['average_confidence']:.3f}")
print("\n  Sentiment Distribution:")
for sentiment, count in summary['sentiment_distribution'].items():
    percentage = (count / summary['total_posts']) * 100
    print(f"    {sentiment:8s}: {count:2d} posts ({percentage:5.1f}%)")

# Visualize results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Sentiment distribution pie chart
sentiment_counts = summary['sentiment_distribution']
colors_pie = ['lightgreen', 'lightcoral', 'lightgray']
ax1.pie(sentiment_counts.values(), labels=sentiment_counts.keys(), autopct='%1.1f%%', 
        colors=colors_pie, startangle=90)
ax1.set_title('😊 Sentiment Distribution')

# Confidence scores distribution
confidences = [r['confidence'] for r in results]
ax2.hist(confidences, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
ax2.set_xlabel('Confidence Score')
ax2.set_ylabel('Frequency')
ax2.set_title('📊 Confidence Score Distribution')

# Timeline of sentiments (by post order)
timeline_sentiments = [r['sentiment'] for r in results]
timeline_colors = ['green' if s == 'POSITIVE' else 'red' if s == 'NEGATIVE' else 'gray' for s in timeline_sentiments]
ax3.scatter(range(len(results)), [1 if s == 'POSITIVE' else -1 if s == 'NEGATIVE' else 0 for s in timeline_sentiments], 
           c=timeline_colors, s=100, alpha=0.7)
ax3.set_xlabel('Post Order')
ax3.set_ylabel('Sentiment')
ax3.set_title('📈 Sentiment Timeline')
ax3.set_yticks([-1, 0, 1])
ax3.set_yticklabels(['Negative', 'Neutral', 'Positive'])
ax3.grid(True, alpha=0.3)

# Word cloud simulation (showing most common words)
all_text = ' '.join([r['text'] for r in results])
words = all_text.lower().split()
# Filter out common words and count frequencies
stop_words = {'the', 'and', 'is', 'it', 'to', 'of', 'in', 'for', 'with', 'on', 'a', 'an'}
word_freq = {}
for word in words:
    word = word.strip('.,!?@#')
    if word not in stop_words and len(word) > 2:
        word_freq[word] = word_freq.get(word, 0) + 1

# Display top words
top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:10]
words_list, counts_list = zip(*top_words) if top_words else ([], [])

ax4.barh(range(len(words_list)), counts_list, color='lightblue')
ax4.set_yticks(range(len(words_list)))
ax4.set_yticklabels(words_list)
ax4.set_xlabel('Frequency')
ax4.set_title('🔤 Most Common Words')

plt.tight_layout()
plt.show()

print("\n💡 Insights:")
positive_pct = (sentiment_counts.get('POSITIVE', 0) / summary['total_posts']) * 100
if positive_pct > 60:
    print(f"   📈 Overall sentiment is very positive ({positive_pct:.1f}% positive posts)")
elif positive_pct > 40:
    print(f"   📊 Sentiment is balanced ({positive_pct:.1f}% positive posts)")
else:
    print(f"   📉 Overall sentiment tends negative ({positive_pct:.1f}% positive posts)")

print(f"   🎯 Model confidence is {'high' if summary['average_confidence'] > 0.8 else 'moderate' if summary['average_confidence'] > 0.6 else 'low'} (avg: {summary['average_confidence']:.3f})")

## 🎓 Chapter 6: Best Practices and Tips

Let's cover some best practices for using TrustformeRS effectively:

In [ ]:
# Best practices demonstration
class TrustformeRSBestPractices:
    
    @staticmethod
    def demonstrate_error_handling():
        """Show proper error handling patterns"""
        print("🛡️ Error Handling Best Practices:")
        print("-" * 40)
        
        # Example: Robust pipeline creation
        try:
            # This would normally be: pipeline("text-classification", "invalid-model")
            print("✅ Always wrap pipeline creation in try-catch blocks")
            print("✅ Provide fallback models if primary model fails")
            print("✅ Log errors with context for debugging")
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
            print("🔄 Falling back to default model...")
    
    @staticmethod
    def demonstrate_memory_management():
        """Show memory management best practices"""
        print("\n💾 Memory Management Best Practices:")
        print("-" * 45)
        
        tips = [
            "🔧 Use appropriate batch sizes (start with 16, adjust based on GPU memory)",
            "📦 Enable model caching for repeated inferences",
            "🗑️ Explicitly clear cache when switching between many models",
            "⚡ Use FP16 precision for inference when accuracy allows",
            "📊 Monitor memory usage in production environments",
            "🔄 Consider model quantization for edge deployment"
        ]
        
        for tip in tips:
            print(f"  {tip}")
    
    @staticmethod
    def demonstrate_performance_optimization():
        """Show performance optimization strategies"""
        print("\n⚡ Performance Optimization Best Practices:")
        print("-" * 50)
        
        optimizations = [
            "🔥 Enable JIT compilation for production workloads",
            "📦 Use batch processing instead of individual requests",
            "💾 Implement result caching for repeated queries",
            "🎯 Choose the right model size for your latency requirements",
            "🔧 Profile your application to identify bottlenecks",
            "🌊 Use streaming for real-time applications",
            "⚖️ Load balance across multiple model instances"
        ]
        
        for opt in optimizations:
            print(f"  {opt}")
    
    @staticmethod
    def demonstrate_production_checklist():
        """Production deployment checklist"""
        print("\n🚀 Production Deployment Checklist:")
        print("-" * 45)
        
        checklist = [
            ("Model Validation", [
                "✅ Test model accuracy on validation dataset",
                "✅ Verify model outputs for edge cases",
                "✅ Check model compatibility with input formats"
            ]),
            ("Performance Testing", [
                "✅ Load test with expected traffic patterns",
                "✅ Measure latency percentiles (P50, P95, P99)",
                "✅ Monitor memory usage under load"
            ]),
            ("Monitoring & Observability", [
                "✅ Set up metrics collection (latency, throughput, errors)",
                "✅ Configure alerting for anomalies",
                "✅ Implement health checks and readiness probes"
            ]),
            ("Security & Compliance", [
                "✅ Validate input sanitization",
                "✅ Implement rate limiting",
                "✅ Ensure data privacy compliance"
            ])
        ]
        
        for category, items in checklist:
            print(f"\n  📋 {category}:")
            for item in items:
                print(f"    {item}")
    
    @staticmethod
    def demonstrate_common_pitfalls():
        """Common pitfalls to avoid"""
        print("\n⚠️ Common Pitfalls to Avoid:")
        print("-" * 35)
        
        pitfalls = [
            "❌ Loading models in request handlers (load once, reuse many times)",
            "❌ Not handling model loading failures gracefully",
            "❌ Using synchronous calls in async applications",
            "❌ Ignoring batch size optimization",
            "❌ Not monitoring resource usage in production",
            "❌ Hardcoding model paths (use configuration files)",
            "❌ Not validating input data before inference",
            "❌ Forgetting to set appropriate timeouts"
        ]
        
        for pitfall in pitfalls:
            print(f"  {pitfall}")

# Run best practices demo
print("🎓 TrustformeRS Best Practices Guide")
print("=" * 40)

best_practices = TrustformeRSBestPractices()
best_practices.demonstrate_error_handling()
best_practices.demonstrate_memory_management()
best_practices.demonstrate_performance_optimization()
best_practices.demonstrate_production_checklist()
best_practices.demonstrate_common_pitfalls()

print("\n🌟 Key Takeaways:")
print("-" * 20)
print("1. 🛡️ Always implement proper error handling and fallbacks")
print("2. ⚡ Optimize for your specific use case (speed vs accuracy vs memory)")
print("3. 📊 Monitor and measure everything in production")
print("4. 🔧 Start simple, then optimize based on real performance data")
print("5. 🚀 TrustformeRS provides the tools - use them wisely!")

## 🎉 Conclusion

Congratulations! You've completed the comprehensive TrustformeRS tutorial. You've learned:

### ✅ What You've Accomplished

1. **Basic Pipeline Usage** - Creating and using text classification, generation, and Q&A pipelines
2. **Advanced Features** - Configuration, streaming, and batch processing
3. **Performance Optimization** - JIT compilation, caching, and profiling
4. **Model Comparison** - Selecting the right model for your needs
5. **Real-World Applications** - Social media sentiment analysis
6. **Best Practices** - Production-ready deployment strategies

### 🚀 Next Steps

1. **Try the Examples** - Run the example scripts in the TrustformeRS repository
2. **Build Your Own Application** - Start with a simple use case and expand
3. **Join the Community** - Contribute to TrustformeRS development
4. **Read the Documentation** - Dive deeper into specific features
5. **Share Your Experience** - Help others learn TrustformeRS

### 📚 Additional Resources

- **GitHub Repository**: [https://github.com/trustformers/trustformers](https://github.com/trustformers/trustformers)
- **Documentation**: [https://trustformers.dev/docs](https://trustformers.dev/docs)
- **Examples**: [https://github.com/trustformers/examples](https://github.com/trustformers/examples)
- **Community**: [https://discord.gg/trustformers](https://discord.gg/trustformers)

### 🤝 Get Involved

TrustformeRS is an open-source project that welcomes contributions:

- **Report Issues**: Found a bug? Let us know!
- **Suggest Features**: Have ideas for improvements?
- **Contribute Code**: Help make TrustformeRS even better
- **Write Documentation**: Share your knowledge with others
- **Create Tutorials**: Help newcomers get started

Thank you for exploring TrustformeRS! 🎉🚀

In [ ]:
# Final celebration! 🎉
print("🎉" * 50)
print("🚀 Congratulations on completing the TrustformeRS tutorial! 🚀")
print("🎉" * 50)
print()
print("You're now ready to build amazing ML applications with TrustformeRS!")
print("Go forth and create something awesome! 🌟")

# Generate a completion certificate
import datetime

completion_date = datetime.datetime.now().strftime("%Y-%m-%d")
certificate = f"""
┌─────────────────────────────────────────────────────────────┐
│                    🏆 CERTIFICATE OF COMPLETION 🏆         │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│              TrustformeRS Interactive Tutorial              │
│                                                             │
│  This certifies that the holder has successfully completed │
│  the comprehensive TrustformeRS tutorial and is now        │
│  qualified to build high-performance ML applications       │
│  using the TrustformeRS framework.                        │
│                                                             │
│  Date: {completion_date:20s}                                │
│                                                             │
│  🚀 Happy coding with TrustformeRS! 🦀                     │
└─────────────────────────────────────────────────────────────┘
"""

print(certificate)